# 画像ファイルに対応するキロ程紐づけ用のJSONファイルを作成するノートブック
```
以下のコマンドを順番に実行していく
想定しているディレクトリ構造は以下のとおり
.
├ imgs
│  └ dir_area    :線区単位の画像を保存
│     ├ HD11     :カメラ毎の画像を保存
│     ├ HD12     :以下、同じ
│     ├ HD21         
│     ├ HD21 
│     ├ HD31 
│     └ HD32 
├ TDM
│  └ TDM_GazoFileIndex.xlsx  :車モニから出力したマスターデータ
└ imgKiro.ipynb  :本ファイル
```

In [ ]:
!pip list

# 一括で線区フォルダごとのJSONファイルを作成する

## 1. ライブラリ読込み＆関数の定義

In [3]:
import importlib
from pprint import pprint
import boto3
import gc
import re
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image, ImageDraw, ImageFont
from src.config import appProperties
import src.helpers as helpers
# import src.visualize as vis
importlib.reload(helpers)
# importlib.reload(vis)

config = appProperties('config.yml')    # config を取得


def get_df(bucket_name, csv_file, columns):
    """ 車モニのマスタデータを読み込む
    Args:
        tdm_fpath(str): 車モニ マスターデータのパス
    Return:
        df_tdm(DataFrame): 読み込まれたデータフレーム
    """

    # TDM(車モニCSV)をS3からダウンロードする
    download_path = f"TDM/temp/{csv_file.split('/')[-1]}"
    print("車モニのマスターデータを読み込みます ※少し時間がかかります")
    download_file_from_s3(bucket_name, csv_file, download_path)
    print(f'complete: {download_path=}')

    # CSVからデータフレームを読み込む
    df_tdm = pd.read_csv(download_path, names=columns, delimiter='|')

    # データフレームを整形する
    df_tdm['TimeCode'] = pd.to_datetime(df_tdm['TimeCode'])
    df_tdm = df_tdm.sort_values(by=['SokuteiDate', 'KiroTei'], ignore_index=True)
    print("車モニのマスターデータを読み込みました")
    print(f"データフレームのサイズ:{df_tdm.shape}")

    # 欲しい列だけ抽出する
    df_tdm = df_tdm.filter([
        'EkiCd', 'SenbetsuCd', 'SokuteiDate', 'DenchuNo', 'KiroTei',               # Comment Out    'SokuteiYear' 'NennaiSeqNo'は追加されている
        'GazoFileNameHD11', 'GazoFileNameHD12',
        'GazoFileNameHD21', 'GazoFileNameHD22',
        'GazoFileNameHD31', 'GazoFileNameHD32'
    ]).copy()

    print("必要な情報だけフィルタリングしました")
    print(f"データフレームのサイズ:{df_tdm.shape}")
    return df_tdm


def list_csv_files(bucket_name, prefix):
    """ S3にあるCSVの一覧をゲット
    """
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')

    csv_files = []
    for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
        if 'Contents' in page:
            for obj in page['Contents']:
                if obj['Key'].endswith('.csv'):
                    csv_files.append(obj['Key'])

    return csv_files


def download_file_from_s3(bucket_name, key, download_path):
    """ S3からダウンロードする
    """
    s3 = boto3.client('s3')
    s3.download_file(bucket_name, key, download_path)


def get_columns_from_csv(bucket_name, columns_csv_key):
    """ S3からカラム名リストをゲット
    """
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket_name, Key=columns_csv_key)
    columns_df = pd.read_csv(obj['Body'])
    return columns_df.columns.tolist()


def get_KiroTei_dict(config, df):
    """ 走行日・線区ごとのキロ程補正情報を辞書に記録する
    """
    # カメラ番号ごとに、EkiCdのはじめの行をTrueとして記録する
    for camera_num in config.camera_types:
        df[f"EkiCdDiff{camera_num}"] = df.query(f"GazoFileName{camera_num}.notnull()", engine='python')['EkiCd'].diff().map(lambda x: x != 0)
    # 走行日・線区ごとに、画像1枚あたりの幅(m)を算出して辞書に記録する
    result_dict = {}
    for camera_num in config.camera_types:
        grouped_df = df.query(f"GazoFileName{camera_num}.notnull()", engine='python').groupby(['SokuteiDate', 'EkiCd'])['KiroTei'].count()
        for (date, ekicd), count in grouped_df.items():
            if date not in result_dict:
                result_dict[date] = {}
            if ekicd not in result_dict[date]:
                result_dict[date][ekicd] = {}
            # print(f"camera_num: {camera_num}, date: {date}, ekicd: {ekicd}")
            KiroTei_head = df.query(f"EkiCdDiff{camera_num}.notnull() & SokuteiDate == {date} & EkiCd == {ekicd}", engine='python')['KiroTei'].iloc[0]
            KiroTei_tail = df.query(f"EkiCdDiff{camera_num}.notnull() & SokuteiDate == {date} & EkiCd == {ekicd}", engine='python')['KiroTei'].iloc[-1]
            result_dict[date][ekicd][camera_num] = {
                'KiroTei_head': KiroTei_head,
                'KiroTei_tail': KiroTei_tail,
                'KiroTei_dist': round(KiroTei_tail - KiroTei_head, 4),
                'KiroTei_delta': round((KiroTei_tail - KiroTei_head) / (count - 1), 6),
                'count': count
            }
    return df.copy(), result_dict


# def get_Kiro_offset(df_tdm, ref_point, date):                                                                                               # 2024.10.4 キロ程オフセット未使用化のためコメントアウト
#     """ 基準位置を指定し、検測キロ程をNEWSSキロ程に変換するオフセット値を取得する
#     Args:
#         df_tdm(pd.DataFrame): 
#     Return:
    
#     """
#     pole_num_matched = df_tdm.query(
#         f"EkiCd == {ref_point['EkiCd_NEWSS']} & SokuteiDate == {date} & DenchuNo == {ref_point['pole_num_NEWSS']}", engine='python'
#     ).index.tolist()
#     pole_kiro_twins = df_tdm.loc[min(pole_num_matched)]['KiroTei']
#     pole_kiro_offset = ref_point['pole_kilo_NEWSS'] - pole_kiro_twins
#     print(f"基準にする電柱の検測キロ程: {pole_kiro_twins}km")
#     print(f"　　　キロ程のオフセット値: {pole_kiro_offset}km")
#     return pole_kiro_offset


# def get_Kiro_NEWSS(df_tdm, kiro_offset_dict):                                                                                               # 2024.10.4 キロ程オフセット未使用化のためコメントアウト
#     df_tdm['KiroTei_NEWSS'] = df_tdm.apply(lambda row: row['KiroTei'] + kiro_offset_dict[row['SokuteiDate']], axis=1)
#     return df_tdm


def set_imgKiro(config, dir_area, df_tdm):
    imgKilo = {}
    for camera_num in tqdm(config.camera_types):
        # カメラフォルダ内の画像ファイルを取得
        image_dir = f"{config.image_dir}/{dir_area}/{camera_num}/"
        print(image_dir)
        list_images = helpers.list_images(image_dir)
        print(f"Image counts:{len(list_images)}")
        # 画像ファイル名に対応するキロ程を抽出して辞書型で記録する
        imgKilo_temp = {}
        # imgKilo_temp_values = {}
        for fname in list_images:
        # for fname in list_images:
            image_name = re.split('[./]', fname)[-2]
            df_tdm_Series = df_tdm[df_tdm[f"GazoFileName{camera_num}"] == image_name].copy()
            if df_tdm_Series.empty:
                continue
            else:
                DenchuNo = df_tdm_Series['DenchuNo'].item()
                # KiroTei = df_tdm_Series['KiroTei_NEWSS'].round(4).item()                            # 2024.10.4 キロ程オフセット未使用化のためコメントアウト
                KiroTei = df_tdm_Series['KiroTei'].round(4).item()                                    # 2024.10.4 キロ程オフセット未使用化のため追加

            # imgKilo_temp_values["DenchuNo"] = DenchuNo
            # imgKilo_temp_values["KiroTei"] = KiroTei
            # imgKilo_temp[image_name] = imgKilo_temp_values.copy()
            imgKilo_temp[image_name] = {
                "DenchuNo": DenchuNo,
                "KiroTei": KiroTei
            }

        if imgKilo_temp != {}:
            imgKilo[camera_num] = imgKilo_temp.copy()

    return imgKilo

print("準備完了")

準備完了


## 駅・駅間コードのjson作成

In [ ]:
## 2024.10.4 キロ程オフセット未使用化のためコメントアウト ##

# エクセル読み込み
# df_ekicode = pd.read_excel(f"{config.tdm_dir}/Senku-Code.xlsx")

# # json元データ作成
# eki_code = {}
# for Senmei in df_ekicode["Senmei"].unique():
#     eki_code[Senmei] = {}
#     # for idx, Kukan in enumerate(df_ekicode["Kukan"]):
#     for idx, Kukan in enumerate(df_ekicode[(df_ekicode['Senmei'] == Senmei)]["Kukan"]):
#         df_ekicode_filtered = df_ekicode.query(f"Senmei == '{Senmei}' & Kukan == '{Kukan}'", engine="python")
#         eki_code[Senmei][Kukan] = {
#             "EkiCd_NEWSS":df_ekicode_filtered["EkiCd"].iloc[0],
#             "pole_num_NEWSS":df_ekicode_filtered["DenchuNo"].iloc[0],
#             "pole_kilo_NEWSS":df_ekicode_filtered["KiroTei"].iloc[0]
#         }

# # jsonファイルに書き出し
# path = f"{config.tdm_dir}/eki_code.json"
# with open(path, mode="wt", encoding="utf-8") as f:
#     json.dump(eki_code, f, ensure_ascii=False, indent=2, default=helpers.default)

In [ ]:
## 2024.10.4 キロ程オフセット未使用化のためコメントアウト ##

# jsonファイル確認（）
# path = path = f"{config.tdm_dir}/eki_code.json"
# with open(path, encoding="utf-8") as f:
#     eki_code_rd =  json.load(f)

# for key in eki_code_rd["Takasaki"].keys():
#     print(key)

In [ ]:
## 2024.10.4 キロ程オフセット未使用化のためコメントアウト ##

# 駅コードのjsonファイルを読み込んでセットする
# path = f"{config.tdm_dir}/eki_code.json"
# with open(path, encoding="utf-8") as f:
#     eki_code =  json.load(f)

## 2. 車モニデータを利用して画像ごとのキロ程マスターデータ(JSON)を作成する
```
ファイルパスなどを設定する
車モニのマスターデータは検測キロ程になっているため、NEWSSキロ程に合わせる基準を設定する
```

In [2]:
# S3にあるCSVファイルを読み込む
bucket_name = 'trolley-monitor'
prefix = 'img-location-table/2024Q1/2024_0001-0180/'
csv_files = list_csv_files(bucket_name, prefix)
print("参考")
print(f"S3にあるTDM(車モニCSV):")
for i, csv_file in enumerate(csv_files):
    print(f"{i}> {csv_file}")

2024-10-07 03:41:56.814 INFO    botocore.credentials: Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


参考
S3にあるTDM(車モニCSV):
0> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0001.csv
1> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0002.csv
2> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0003.csv
3> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0004.csv
4> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0005.csv
5> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0006.csv
6> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0007.csv
7> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0008.csv
8> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0009.csv
9> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0010.csv
10> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0011.csv
11> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0012.csv
12> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_20

### imgs内の全ての線区ディレクトリに対して、キロ程jsonを作成する

In [4]:
%%time

# 初期設定
images_path = helpers.list_imagespath_nonCache(config.image_dir)

# カラム名をS3から読み込む
columns_csv_key = 'img-location-table/TDM_GazoFileIndex_ColumnsName.csv'
columns = get_columns_from_csv(bucket_name, columns_csv_key)
# print(f"{columns=}")

# meas_list = []
for images_folder_num in images_path:
    # カメラ番号のリスト
    cams_path = helpers.list_imagespath_nonCache(f"{config.image_dir}/{images_folder_num}")

    # 線区フォルダ内の最初の画像ファイル
    target_dir = f"{config.image_dir}/{images_folder_num}/{cams_path[0]}"
    base_images = helpers.list_images(target_dir)
    image_name = base_images[0].split('/')[-1]
    print(f"{image_name=}")

    meas_year = image_name.split("_")[0]              # 走行年度
    meas_idx = image_name.split("_")[1]               # 年度通番
    meas_senku = images_folder_num.split('_')[0]      # 線区
    meas_kukan = images_folder_num.split('_')[2]      # 区間
    print(f"測定年度: {meas_year}  年度通番: {meas_idx}  線区: {meas_senku}  区間: {meas_kukan}")

    # 手持ち線区フォルダと一致するCSVをS3から探す
    # stop_loop = 0
    for csv_file in csv_files:
        if csv_file[-13:-4] == f"{meas_year}_{meas_idx}":
            # データフレームを読み込む
            df_tdm = get_df(bucket_name, csv_file, columns)

            # 走行日・線区ごとのキロ程補正情報の辞書を作成する
            df_tdm, KiroTei_dict = get_KiroTei_dict(config, df_tdm)

            # 測定日を取得　※１つしかないはず・・・
            date = df_tdm['SokuteiDate'].unique().item()

            # 線別コードを取得
            if images_folder_num.split('_')[3] == "down":
                SenbetsuCd = 21
            elif images_folder_num.split('_')[3] == "up":
                SenbetsuCd = 22
            else:
                SenbetsuCd = None                              # 他は？？

            # 基準となる電柱の情報                                                                                               # キロ程オフセット未使用化のため追加
            # ref_point = {
            #     "EkiCd_NEWSS": eki_code[meas_senku][meas_kukan]["EkiCd_NEWSS"],
            #     "pole_num_NEWSS": eki_code[meas_senku][meas_kukan]["pole_num_NEWSS"],
            #     "pole_kilo_NEWSS": eki_code[meas_senku][meas_kukan]["pole_kilo_NEWSS"]
            # }
            
            # print('df_tdm <head>')
            # print(df_tdm.head())

            # pole_kiro_offset = get_Kiro_offset(df_tdm, ref_point, date)                                                        # キロ程オフセット未使用化のため追加

            # kiro_offset_dict = {}                                                                                              # キロ程オフセット未使用化のため追加
            # kiro_offset_dict[date] = pole_kiro_offset                                                                          # キロ程オフセット未使用化のため追加

            # # NEWSSキロ程を計算して 列`KiroTei_NEWSS`に記録する                                                                  # キロ程オフセット未使用化のため追加
            # df_tdm = get_Kiro_NEWSS(df_tdm, kiro_offset_dict)                                                                  # キロ程オフセット未使用化のため追加

            # 確認用にエクセルに出力する
            df_tdm.to_excel(f"./TDM/temp/df_tdm_{images_folder_num}.xlsx")

            # 初期設定の情報を出力
            print(f"画像フォルダ名：{images_folder_num}")
            # print(f"基準にする電柱：{ref_point['pole_num_NEWSS']}")                                                             # キロ程オフセット未使用化のため追加
            # print(f"　　　　キロ程：{ref_point['pole_kilo_NEWSS']}")                                                            # キロ程オフセット未使用化のため追加
            # print(f"検測キロ程ズレ：{round(kiro_offset_dict[date], 3)}")                                                        # キロ程オフセット未使用化のため追加

            imgKilo = set_imgKiro(config, images_folder_num, df_tdm)

            if imgKilo != {}:
                # 結果をJSONファイルに記録する
                dir = f"{config.tdm_dir}/{images_folder_num}.json"
                with open(dir, mode="wt", encoding="utf-8") as f:
                    json.dump(imgKilo, f, ensure_ascii=False, indent=2, default=helpers.default)

            print(f"画像ファイルごとのキロ程情報を{dir}に記録しました")
            break

#     if stop_loop == 1:
#         break


"""
2024.8.12　めも
・必要なCSVファイルのみS3から持ってくる形式に変更（S3にある場合のみ）
・必要なCSVファイルは走行年度と年度内通番から検索
・それを手持ちの線区フォルダで順繰りに実施
"""

print("線区ディレクトリにある画像ファイルに対応するキロ程情報を読み込みました")

image_name='2024_0004_HD11_01_00020303.jpg'
測定年度: 2024  年度通番: 0004  線区: Joban  区間: Toride-st
車モニのマスターデータを読み込みます ※少し時間がかかります
complete: download_path='TDM/temp/TDM_GazoFileIndex_2024_0004.csv'


<timed exec>:31: DtypeWarning: Columns (23,26,29,32) have mixed types.Specify dtype option on import or set low_memory=False.


車モニのマスターデータを読み込みました
データフレームのサイズ:(83501, 90)
必要な情報だけフィルタリングしました
データフレームのサイズ:(83501, 11)
画像フォルダ名：Joban_001_Toride-st_down_20240402_day


  0%|          | 0/6 [00:00<?, ?it/s]

imgs/Joban_001_Toride-st_down_20240402_day/HD11/
Image counts:518
imgs/Joban_001_Toride-st_down_20240402_day/HD12/
Image counts:518
imgs/Joban_001_Toride-st_down_20240402_day/HD21/
Image counts:518
imgs/Joban_001_Toride-st_down_20240402_day/HD22/
Image counts:518
imgs/Joban_001_Toride-st_down_20240402_day/HD31/
Image counts:518
imgs/Joban_001_Toride-st_down_20240402_day/HD32/
Image counts:518
画像ファイルごとのキロ程情報をTDM/Joban_001_Toride-st_down_20240402_day.jsonに記録しました
image_name='2024_0003_HD11_01_00042619.jpg'
測定年度: 2024  年度通番: 0003  線区: Joban  区間: Toride-st
車モニのマスターデータを読み込みます ※少し時間がかかります
complete: download_path='TDM/temp/TDM_GazoFileIndex_2024_0003.csv'
車モニのマスターデータを読み込みました
データフレームのサイズ:(138202, 90)
必要な情報だけフィルタリングしました
データフレームのサイズ:(138202, 11)
画像フォルダ名：Joban_001_Toride-st_up_20240402_day


...

imgs/Yokohama_008_ShinYokohama-Kozukue_up_20240405_day/HD11/
Image counts:594
imgs/Yokohama_008_ShinYokohama-Kozukue_up_20240405_day/HD12/
Image counts:594
imgs/Yokohama_008_ShinYokohama-Kozukue_up_20240405_day/HD21/
Image counts:594
imgs/Yokohama_008_ShinYokohama-Kozukue_up_20240405_day/HD22/
Image counts:594
imgs/Yokohama_008_ShinYokohama-Kozukue_up_20240405_day/HD31/
Image counts:593
imgs/Yokohama_008_ShinYokohama-Kozukue_up_20240405_day/HD32/
Image counts:593
画像ファイルごとのキロ程情報をTDM/Yokohama_008_ShinYokohama-Kozukue_up_20240405_day.jsonに記録しました
image_name='2024_0015_HD11_01_00003744.jpg'
測定年度: 2024  年度通番: 0015  線区: Yokohama  区間: Kozukue-st
車モニのマスターデータを読み込みます ※少し時間がかかります
complete: download_path='TDM/temp/TDM_GazoFileIndex_2024_0015.csv'
車モニのマスターデータを読み込みました
データフレームのサイズ:(53859, 90)
必要な情報だけフィルタリングしました
データフレームのサイズ:(53859, 11)
画像フォルダ名：Yokohama_009_Kozukue-st_down_20240405_day


  0%|          | 0/6 [00:00<?, ?it/s]

imgs/Yokohama_009_Kozukue-st_down_20240405_day/HD11/
Image counts:552
imgs/Yokohama_009_Kozukue-st_down_20240405_day/HD12/
Image counts:552
imgs/Yokohama_009_Kozukue-st_down_20240405_day/HD21/
Image counts:552
imgs/Yokohama_009_Kozukue-st_down_20240405_day/HD22/
Image counts:552
imgs/Yokohama_009_Kozukue-st_down_20240405_day/HD31/
Image counts:552
imgs/Yokohama_009_Kozukue-st_down_20240405_day/HD32/
Image counts:552
画像ファイルごとのキロ程情報をTDM/Yokohama_009_Kozukue-st_down_20240405_day.jsonに記録しました
image_name='2024_0014_HD11_01_00017673.jpg'
測定年度: 2024  年度通番: 0014  線区: Yokohama  区間: Kozukue-st
車モニのマスターデータを読み込みます ※少し時間がかかります
complete: download_path='TDM/temp/TDM_GazoFileIndex_2024_0014.csv'
車モニのマスターデータを読み込みました
データフレームのサイズ:(52317, 90)
必要な情報だけフィルタリングしました
データフレームのサイズ:(52317, 11)
画像フォルダ名：Yokohama_009_Kozukue-st_up_20240405_day


  0%|          | 0/6 [00:00<?, ?it/s]

imgs/Yokohama_009_Kozukue-st_up_20240405_day/HD11/
Image counts:570
imgs/Yokohama_009_Kozukue-st_up_20240405_day/HD12/
Image counts:570
imgs/Yokohama_009_Kozukue-st_up_20240405_day/HD21/
Image counts:570
imgs/Yokohama_009_Kozukue-st_up_20240405_day/HD22/
Image counts:570
imgs/Yokohama_009_Kozukue-st_up_20240405_day/HD31/
Image counts:570
imgs/Yokohama_009_Kozukue-st_up_20240405_day/HD32/
Image counts:570
画像ファイルごとのキロ程情報をTDM/Yokohama_009_Kozukue-st_up_20240405_day.jsonに記録しました
線区ディレクトリにある画像ファイルに対応するキロ程情報を読み込みました
CPU times: user 21min 51s, sys: 11.5 s, total: 22min 3s
Wall time: 27min 24s


## 作成したJSONファイルの確認

In [ ]:
import json
jsonPath = config.tdm_dir
# jsonFile = "Joban_001_Toride-st_down_20240402_day.json"
jsonFile = "Joban_002_Toride-Fujishiro_down_20240402_day.json"
FullPath = jsonPath + "/" + jsonFile
print(FullPath)

jsonOpen = open(FullPath, 'r')
tdmFile = json.load(jsonOpen)

# print(jsonLoad)

In [ ]:
jsonFile.split('.')[0]

In [ ]:
import pandas as pd
from openpyxl import Workbook
# import openpyxl

# writer = pd.ExcelWriter('Takasaki_99_Omiya-Kitaageo_down_20230509_day.xlsx', engine='openpyxl')
# wb = openpyxl.Workbook()

jsonname = jsonFile.split('.')[0]

with pd.ExcelWriter(f'{jsonname}.xlsx') as writer:

    for camera in ['HD11', 'HD12', 'HD21', 'HD22', 'HD31', 'HD32']:

        Index, Kirotei, KiroDiff, PoleNum, NewPole = [], [], [], [], []
        for ix, file in enumerate(tdmFile[camera]):
            if ix == 0:
                last_pole = tdmFile[camera][file]['DenchuNo']
                last_kiro = tdmFile[camera][file]['KiroTei']

            Index.append(ix)

            Kirotei.append(tdmFile[camera][file]['KiroTei'])

            KiroDiff.append(tdmFile[camera][file]['KiroTei'] - last_kiro)

            PoleNum.append(tdmFile[camera][file]['DenchuNo'])

            if tdmFile[camera][file]['DenchuNo'] != last_pole:
                NewPole.append('True')
            else:
                NewPole.append('False')

            last_pole = tdmFile[camera][file]['DenchuNo']
            last_kiro = tdmFile[camera][file]['KiroTei']

        df_json = pd.DataFrame({'Index': Index, 'PoleNum': PoleNum, 'NewPole': NewPole, 'Kirotei': Kirotei, 'KiroDiff':KiroDiff})
        df_json.to_excel(writer, sheet_name=camera, index=False)
        # df_json.to_excel(wb, sheet_name=camera, index=False)

# Excelファイルを保存
# writer.save()
# wb.save('Takasaki_99_Omiya-Kitaageo_down_20230509_day.xlsx')


In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
output_notebook()

source = ColumnDataSource(df_json)

TOOLTIPS = [
    ('Index', '@ix'),
    ('PoleNum', '@PoleNum'),
    ('Kirotei', '@Kirotei')
]

p = figure(
    plot_width=2000, 
    plot_height=600,
    tooltips=TOOLTIPS
)
     
p.circle(
    source=source, 
    # x='ix',
    # y='kiro'
    x='Index',
    y='Kirotei'
)

show(p)

In [ ]:
date = df_tdm['SokuteiDate'].unique().item()
date

In [ ]:
meas_senku = 'Joban'
meas_kukan = 'Toride-st'

ref_point = {
    "EkiCd_NEWSS": eki_code[meas_senku][meas_kukan]["EkiCd_NEWSS"],
    "pole_num_NEWSS": eki_code[meas_senku][meas_kukan]["pole_num_NEWSS"],
    "pole_kilo_NEWSS": eki_code[meas_senku][meas_kukan]["pole_kilo_NEWSS"]
}

In [ ]:
ref_point

In [ ]:
pole_num_matched = df_tdm.query(
    f"EkiCd == {ref_point['EkiCd_NEWSS']} & SokuteiDate == {date} & DenchuNo == {ref_point['pole_num_NEWSS']}", engine='python'
).index.tolist()

In [ ]:
df_tdm[df_tdm['EkiCd'] == ref_point['EkiCd_NEWSS']]

In [ ]:
pole_num_matched

In [ ]:
df_tdm